## Preprocessing

### Import libraries 

In [2]:
!pip install scikit-image
!pip install --upgrade scikit-learn

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from skimage.io import imread
from skimage.transform import resize

### Load and processing images

In [30]:
# Load metadata
metadata = pd.read_csv("E:/Coding/ML/Dataset/LFW/lfw_allnames.csv")
people_train_metadata = pd.read_csv("E:/Coding/ML/Dataset/LFW/peopleDevTrain.csv")
people_test_metadata = pd.read_csv("E:/Coding/ML/Dataset/LFW/peopleDevTest.csv")

In [37]:
# Function to load and preprocess images in batches
def load_images_with_metadata(directory, image_size, metadata, batch_size=32):
    X = []
    y = []
    for subdir, _, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(subdir, filename)
            if os.path.isfile(filepath):
                image = imread(filepath, as_gray=True)
                resized_image = resize(image, image_size)
                flattened_image = resized_image.flatten()
                X.append(flattened_image)
                label = os.path.basename(os.path.dirname(filepath))
                y.append(label)
                if len(X) == batch_size:
                    yield np.array(X), np.array(y)
                    X, y = [], []
    if X:
        yield np.array(X), np.array(y)

In [38]:
X_train = []
y_train = []
for X_batch, y_batch in load_images_with_metadata(directory='E:/Coding/ML/Dataset/LFW/lfw-deepfunneled', 
                                                   image_size=(50, 50), 
                                                   metadata=people_train_metadata):
    X_train.extend(X_batch)
    y_train.extend(y_batch)

# Load and preprocess testing images with metadata
X_test = []
y_test = []
for X_batch, y_batch in load_images_with_metadata(directory='E:/Coding/ML/Dataset/LFW/lfw-deepfunneled', 
                                                  image_size=(50, 50), 
                                                  metadata=people_test_metadata):
    X_test.extend(X_batch)
    y_test.extend(y_batch)


### NAIVE BAYES SKLEARN MODEL


In [39]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

GaussianNB()

### Predict on the test set

In [40]:
y_pred = nb_classifier.predict(X_test)

MemoryError: Unable to allocate 252. MiB for an array with shape (13233, 2500) and data type float64

### Evaluation

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_pred, average='weighted')  # 'weighted' takes into account class imbalance
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_pred, average='weighted')  # 'weighted' takes into account class imbalance
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # 'weighted' takes into account class imbalance
print("F1 Score:", f1)

Accuracy: 0.03853418964865886
Precision: 0.034463692831338724
Recall: 0.03853418964865886
F1 Score: 0.02671705264216078


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
